In [ ]:
# server
import socket
import threading
import json
import base64
from cryptography.fernet import Fernet

# Generate a key for encryption/decryption
KEY = Fernet.generate_key()
cipher = Fernet(KEY)

def save_key():
    with open("secret.key", "wb") as key_file:
        key_file.write(KEY)

def load_key():
    with open("secret.key", "rb") as key_file:
        return key_file.read()

# Ensure key is saved and loaded properly
save_key()
KEY = load_key()
cipher = Fernet(KEY)

# User database (for authentication)
USER_CREDENTIALS = {
    "user1": "password1",
    "user2": "password2"
}

# Dictionary to track authenticated users
connected_users = {}

# UDP Server
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))
print("UDP Chat Server is running...")

# Function to handle incoming client messages
def handle_client(data, client_address):
    global connected_users
    
    try:
        message = json.loads(data.decode())  # Decode and parse the received JSON message
        
        if message["type"] == "auth":  # Handle authentication requests
            username = message["username"]
            password = message["password"]
            
            if username in USER_CREDENTIALS and USER_CREDENTIALS[username] == password:
                connected_users[client_address] = username  # Store authenticated user
                response = json.dumps({"status": "success", "message": "Authentication successful."})
            else:
                response = json.dumps({"status": "error", "message": "Invalid credentials."})
                
            server_socket.sendto(response.encode(), client_address)  # Send authentication response
        
        
        elif message["type"] == "message":  # Handle chat messages
            if client_address in connected_users:   # Ensure user is authenticated
                decrypted_msg = cipher.decrypt(base64.urlsafe_b64decode(message["content"].encode())).decode()
                print(f"{connected_users[client_address]}: {decrypted_msg}")  # Display received message
            else:
                response = json.dumps({"status": "error", "message": "Unauthorized user."})
                server_socket.sendto(response.encode(), client_address)  # Send error message if unauthorized
    
    except Exception as e:
        print("Error handling message:", e)

# Listening for clients
while True:
    data, client_address = server_socket.recvfrom(4096)
    threading.Thread(target=handle_client, args=(data, client_address)).start()


UDP Chat Server is running...
user1: this is message from the past
user1: I am still here
user1: 
